# SW-7b-Python-OWL

**Sidetrack Python** : [<< SW-7-CSharp-OWL](SW-7-CSharp-OWL.ipynb) | [Index](README.md) | [SW-8-Python-SHACL >>](SW-8-Python-SHACL.ipynb)

## Ontologies OWL en Python avec OWLReady2

Ce notebook est un **sidetrack optionnel** qui presente l'equivalent Python des concepts OWL du notebook SW-7 (dotNetRDF). Vous y decouvrirez **OWLReady2**, une bibliotheque Python pour manipuler des ontologies OWL et effectuer du raisonnement.

### Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Creer et manipuler des ontologies OWL avec OWLReady2
2. Definir des classes, proprietes et restrictions
3. Utiliser un raisonneur (HermiT) pour deduire de nouvelles connaissances
4. Faire la correspondance entre dotNetRDF et OWLReady2

### Prerequis
- SW-7-CSharp-OWL recommande (pour la comprehension conceptuelle)
- Python 3.10+
- Java requis pour le raisonneur HermiT (inclus dans OWLReady2)

### Duree estimee : 30 minutes

> **Note** : OWLReady2 inclut le raisonneur HermiT (Java) pour le raisonnement OWL 2 DL complet.

---

## 1. Installation et Imports

**OWLReady2** est la bibliotheque de reference pour manipuler des ontologies OWL en Python.

In [ ]:
%pip install -q owlready2

In [ ]:
from owlready2 import *
import owlready2

print("OWLReady2 importe.")
print(f"Version : {owlready2.__version__}")

---

## 2. Creer une Ontologie Simple

Commencons par creer une ontologie pour une famille de personnes, avec des classes et des proprietes.

In [ ]:
# Create a new ontology
onto = get_ontology("http://example.org/family.owl#")

# Define classes
with onto:
    class Person(Thing):
        pass
    
    class Man(Person):
        pass
    
    class Woman(Person):
        pass
    
    # Properties
    class has_child(ObjectProperty, FunctionalProperty):
        domain = [Person]
        range = [Person]
        inverse_property = has_parent
    
    class has_parent(ObjectProperty):
        domain = [Person]
        range = [Person]
    
    class has_name(DataProperty, FunctionalProperty):
        domain = [Person]
        range = [str]
    
    class age(DataProperty, FunctionalProperty):
        domain = [Person]
        range = [int, float]

print("Ontologie creee avec classes :")
print(f"  Person (Thing subclass)")
print(f"  Man (Person subclass)")
print(f"  Woman (Person subclass)")
print()
print("Proprietes :")
print(f"  has_child (ObjectProperty, inverse: has_parent)")
print(f"  has_name (DataProperty, string)")
print(f"  age (DataProperty, integer)")

### Interpretation : Creation d'ontologie

**Syntaxe OWLReady2** :
- `Thing` est la classe racine OWL (equivalent de `owl:Thing`)
- `ObjectProperty` relie deux individus (comme `owl:ObjectProperty`)
- `DataProperty` relie un individu a une valeur litterale (comme `owl:DatatypeProperty`)
- `FunctionalProperty` signifie qu'une entite ne peut avoir qu'une seule valeur (max 1)

| OWLReady2 | OWL (Turtle) | dotNetRDF |
|-----------|-------------|----------|
| `class Person(Thing)` | `:Person a owl:Class ; rdfs:subClassOf owl:Thing .` | `new OntologyClass("Person")` |
| `ObjectProperty` | `a owl:ObjectProperty` | `new OntologyObjectProperty("has_child")` |
| `DataProperty` | `a owl:DatatypeProperty` | `new OntologyDataProperty("age")` |
| `domain = [Person]` | `rdfs:domain :Person` | `.Domain = personClass` |
| `range = [Person]` | `rdfs:range :Person` | `.Range = personClass` |

---

## 3. Creer des Individus et Assertions

Ajoutons des individus (instances) a notre ontologie.

In [ ]:
# Create individuals
john = Man("John", has_name="John Smith", age=45)
mary = Woman("Mary", has_name="Mary Smith", age=42)
alice = Woman("Alice", has_name="Alice Smith", age=15)

# Define relationships
john.has_child.append(alice)
mary.has_child.append(alice)

print("Individus crees :")
print(f"  {john.name} : {john.has_name}, {john.age} ans")
print(f"  {mary.name} : {mary.has_name}, {mary.age} ans")
print(f"  {alice.name} : {alice.has_name}, {alice.age} ans")
print()
print("Relations parent-enfant :")
print(f"  {john.name} -> has_child -> {alice.name}")
print(f"  {mary.name} -> has_child -> {alice.name}")
print()
print("Propriete inverse (auto-generee) :")
print(f"  {alice.name} -> has_parent -> {[p.name for p in alice.has_parent]}")

### Interpretation : Individus

OWLReady2 utilise une syntaxe Python intuitive :
- `Man("John", ...)` cree un individu de la classe `Man` avec l'IRI `#John`
- Les proprietes sont accessibles comme des attributs Python
- Les proprietes inverse sont automatiquement maintenues

| Operation | OWLReady2 | OWL (Turtle) |
|-----------|-----------|-------------|
| Creer individu | `john = Man("John")` | `:John a :Man .` |
| Assert propriete | `john.has_child.append(alice)` | `:John :has_child :Alice .` |
| Propriete data | `john.has_name = "John"` | `:John :has_name "John" .` |

---

## 4. Restrictions OWL

OWL permet d'exprimer des restrictions sur les classes. Creons une nouvelle ontologie avec des restrictions.

In [ ]:
# Create a new ontology with restrictions
onto2 = get_ontology("http://example.org/company.owl#")

with onto2:
    class Person(Thing):
        pass
    
    class Department(Thing):
        pass
    
    class works_in(ObjectProperty):
        domain = [Person]
        range = [Department]
    
    class manages(ObjectProperty):
        domain = [Person]
        range = [Department]
    
    # Restriction: Manager is someone who manages at least one department
    class Manager(Person):
        equivalent_to = [Person & manages.some(Department)]
    
    # Restriction: Employee is someone who works in some department
    class Employee(Person):
        equivalent_to = [Person & works_in.some(Department)]

print("Classes avec restrictions :")
print(f"  Manager = Person AND manages SOME Department")
print(f"  Employee = Person AND works_in SOME Department")
print()
print("Restriction OWLReady2 :")
print(f"  manages.some(Department) = ∃ manages . Department")
print(f"  Person & manages.some(Department) = Person ∩ (∃ manages . Department)")

### Interpretation : Restrictions OWL

| Restriction OWLReady2 | OWL (Manchester) | Turtle | Signification |
|----------------------|-------------------|--------|---------------|
| `manages.some(Department)` | `manages some Department` | `owl:someValuesFrom` | ∃ : au moins 1 |
| `manages.only(Department)` | `manages only Department` | `owl:allValuesFrom` | ∀ : seulement |
| `manages.max(1)` | `manages max 1` | `owl:maxCardinality` | ≤ 1 |
| `manages.min(1)` | `manages min 1` | `owl:minCardinality` | ≥ 1 |
| `manages.exactly(1)` | `manages exactly 1` | `owl:cardinality` | = 1 |

Equivalent dotNetRDF : Restrictions avec `OntologyClass` et expressions OWL.

---

## 5. Raisonnement avec HermiT

OWLReady2 inclut le raisonneur HermiT qui peut deduire de nouvelles connaissances a partir de l'ontologie.

In [ ]:
# Create individuals and run reasoning
with onto2:
    it_dept = Department("IT")
    hr_dept = Department("HR")
    
    bob = Person("Bob")
    alice = Person("Alice")
    charlie = Person("Charlie")
    
    bob.works_in = [it_dept]
    alice.works_in = [hr_dept]
    charlie.manages = [it_dept]
    charlie.works_in = [it_dept]  # Charlie also works in IT

print("=== Avant raisonnement ===")
print(f"Bob types : {bob.is_a}")
print(f"Alice types : {alice.is_a}")
print(f"Charlie types : {charlie.is_a}")
print()

# Run HermiT reasoner
with onto2:
    sync_reasoner()

print("=== Apres raisonnement (HermiT) ===")
print(f"Bob types : {bob.is_a}")
print(f"Alice types : {alice.is_a}")
print(f"Charlie types : {charlie.is_a}")
print()
print("Deductions :")
print(f"  Bob est Employee (car works_in SOME Department)")
print(f"  Alice est Employee (car works_in SOME Department)")
print(f"  Charlie est Employee ET Manager (car works_in ET manages)")

### Interpretation : Raisonnement

Le raisonneur HermiT a deduit :
- Bob est Employee : il travaille dans un departement (IT)
- Alice est Employee : elle travaille dans un departement (HR)
- Charlie est Employee ET Manager : il travaille dans ET dirige un departement

| Operation | OWLReady2 | dotNetRDF |
|-----------|-----------|----------|
| Raisonnement | `sync_reasoner()` (HermiT) | `OntologyGraph` avec `RdfsReasoner` |
| Types inférés | `.is_a` (mis a jour) | `GetTypes()` (inclus inférés) |
| Raisonneur | HermiT (OWL 2 DL complet) | RDFS + OWL 2 RL limite |

> **Note** : HermiT est un raisonneur OWL 2 DL complet, beaucoup plus puissant que le raisonneur RDFS de base de dotNetRDF.

---

## 6. Charger une Ontologie Existante

OWLReady2 peut charger des ontologies depuis des fichiers ou des URLs.

In [ ]:
# Load the university ontology (if available)
import os

ontology_file = "data/university.owl"

if os.path.exists(ontology_file):
    # Load existing ontology
    onto_univ = get_ontology("file://" + os.path.abspath(ontology_file)).load()
    
    print(f"Ontologie chargee : {onto_univ.base_iri}")
    print(f"Classes : {len(list(onto_univ.classes()))}")
    print(f"Individuals : {len(list(onto_univ.individuals()))}")
    print()
    
    # Show some classes
    print("Classes dans l'ontologie :")
    for cls in list(onto_univ.classes())[:10]:
        print(f"  {cls.name}")
else:
    print(f"Fichier {ontology_file} non trouve.")
    print("Utilisation de l'ontologie de demonstration interne.")
    
    # Use our demo ontology
    onto_univ = onto
    print(f"\nOntologie de demonstration : {onto_univ.base_iri}")
    for cls in onto_univ.classes():
        print(f"  {cls.name}")

---

## 7. Exporter une Ontologie

OWLReady2 peut exporter les ontologies dans differents formats.

In [ ]:
# Export ontology to RDF/XML
onto2.save(file="data/temp_company.owl", format="rdfxml")
print("Ontologie exportee : data/temp_company.owl")
print()

# Also export to Turtle
onto2.save(file="data/temp_company.ttl", format="ntriples")
print("Ontologie exportee : data/temp_company.ttl")
print()

# Show first 20 lines of the Turtle export
print("=== Apercu Turtle (20 premieres lignes) ===")
with open("data/temp_company.ttl", "r") as f:
    for i, line in enumerate(f):
        if i >= 20:
            break
        print(line.rstrip())

---

## 8. Tableau de Correspondance dotNetRDF / OWLReady2

Ce tableau recapitule les equivalences pour la manipulation d'ontologies.

| Operation | dotNetRDF (C#) | OWLReady2 (Python) |
|-----------|---------------|---------------------|
| **Creer ontologie** | `new OntologyGraph()` | `get_ontology("uri#")` |
| **Creer classe** | `new OntologyClass("Person")` | `class Person(Thing): pass` |
| **Sous-classe** | `subClassOf` | `class Person(Thing): pass` |
| **ObjectProperty** | `new OntologyObjectProperty("knows")` | `class knows(ObjectProperty): pass` |
| **DataProperty** | `new OntologyDataProperty("age")` | `class age(DataProperty): pass` |
| **Creer individu** | `g.CreateUriNode("ex:John")` + assertions | `john = Person("John")` |
| **Assert propriete** | `g.Assert(new Triple(...))` | `john.knows.append(alice)` |
| **Restriction SOME** | `new SomeValuesRestriction(...)` | `knows.some(Person)` |
| **Restriction ONLY** | `new AllValuesRestriction(...)` | `knows.only(Person)` |
| **Raisonnement** | `new RdfsReasoner().Apply(graph)` | `sync_reasoner()` (HermiT) |
| **Charger fichier** | `FileLoader.Load(graph, file)` | `get_ontology("file://...").load()` |
| **Exporter** | `new CompressingTurtleWriter().Save()` | `onto.save(file="...")` |

### Differences philosophiques

| Aspect | dotNetRDF | OWLReady2 |
|--------|-----------|----------|
| **Style** | Triple-based (bas niveau) | Class-based (haut niveau) |
| **Modificate** | Modifications directes de triples | Python classes + instances |
| **Raisonneur** | RDFS + OWL RL | HermiT (OWL 2 DL complet) |
| **Performance** | Rapide pour gros graphes | Optimise pour ontologies |

> **Choix** : Utilisez dotNetRDF pour des graphes RDF generiques, OWLReady2 pour des ontologies OWL avec raisonnement complexe.

---

## Exercices

Mettez en pratique les concepts OWL avec ces deux exercices.

### Exercice 1 : Famille avec restrictions

Crez une ontologie de famille avec :
- Classes : `Person`, `Parent`, `Child`
- Propriete : `has_child` (inverse : `has_parent`)
- Restrictions : `Parent` = Person qui `has_child` SOME Person

Crez des individus et verifiez que le raisonneur deduit correctement les types.

In [ ]:
# Exercice 1 : Famille avec restrictions

# TODO: Completez l'ontologie
onto_family = get_ontology("http://example.org/family2.owl#")

with onto_family:
    # Define classes
    # class Person(Thing): pass
    # class Parent(Person):
    #     equivalent_to = [Person & has_child.some(Person)]
    # ...
    pass

# Create individuals and test reasoning
# ...

print("Exercice 1 a completer")

### Exercice 2 : Graphe de connaissances de films

Crez une ontologie pour des films :
- Classes : `Movie`, `Person` (acteur/realisateur), `Genre`
- Proprietes : `has_actor`, `has_director`, `has_genre`
- Restrictions : `ActionMovie` = Movie qui `has_genre` SOME Action

Crez un exemple et verifiez le raisonnement.

In [ ]:
# Exercice 2 : Ontologie de films

# TODO: Completez l'ontologie
onto_movies = get_ontology("http://example.org/movies.owl#")

with onto_movies:
    # class Movie(Thing): pass
    # class Person(Thing): pass
    # class Genre(Thing): pass
    # ...
    pass

print("Exercice 2 a completer")

---

## Resume

Ce sidetrack a presente la manipulation d'ontologies OWL en Python avec **OWLReady2**.

### Concepts cles

| Concept | Ce que vous avez appris |
|---------|------------------------|
| **OWLReady2** | Manipuler des ontologies OWL en Python |
| **Classes/Proprietes** | Definir la structure d'une ontologie |
| **Restrictions** | ∃ (some), ∀ (only), cardinalites |
| **HermiT** | Raisonneur OWL 2 DL complet |
| **Correspondance** | Equivalences dotNetRDF / OWLReady2 |

### Prochaines etapes

- **SW-8-Python-SHACL** : Validation de donnees avec pySHACL
- **SW-9-Python-JSONLD** : Donnees structurees pour le web
- **SW-11-Python-KnowledgeGraphs** : Graphes de connaissances avec kglab

---

**Navigation** : [<< SW-7-CSharp-OWL](SW-7-CSharp-OWL.ipynb) | [Index](README.md) | [SW-8-Python-SHACL >>](SW-8-Python-SHACL.ipynb)